In [1]:
import pandas as pd
from ipywidgets import widgets
from IPython.display import display
import datetime as dt
pd.options.display.max_colwidth = 10000
pd.options.display.max_rows = 10000

def append_row(accounts_df, date_str, col_name, value, description):
    row_idx = len(accounts_df)
    date_obj = dt.datetime.strptime(date_str,"%Y-%m-%d 00:00:00")
    accounts_joined.loc[row_idx, "Date"] = date_obj
    accounts_joined.loc[row_idx, "Description"] = description
    accounts_joined.loc[row_idx, col_name] = value

all_payments = pd.read_csv("/Users/johnclark/Documents/Finances/All_Payment_Methods020122.csv")
end_of_year = pd.read_csv("/Users/johnclark/Documents/Finances/Year End All Payment Methods020122.csv",parse_dates=[2], header=0)
end_of_year["Amount_value"] = end_of_year["Amount"].apply(lambda x: float(x[1:].replace(",","")) if x.startswith("$") else -float(x[2:].replace(",","")) if x.startswith("-$") else x)

no_credit_card = end_of_year[end_of_year["Payee"] != "WELLS FARGO CREDIT CARD"]
no_wells_credit_rec_trans = end_of_year[(end_of_year["Payee"] != "WELLS FARGO CREDIT CARD") & (~end_of_year["Description"].str.startswith("RECURRING TRANSFER")) \
                                        & (~end_of_year["Description"].str.startswith("ONLINE TRANSFER TO CLARK J SAVINGS XXXXXX5751")) \
                                        & (~end_of_year["Description"].str.startswith("ONLINE TRANSFER TO CLARK J CHECKING XXXXXX9873"))]

years = [2020, 2021, 2022]
accounts_joined = pd.DataFrame([], columns=["Date","Amount","Unk","CheckNum","Description"])
for year in years:
    utility_check = pd.read_csv("/Users/johnclark/Documents/Finances/Utility{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
    utility_check["account_name"] = "utility"
    umbrella_check = pd.read_csv("/Users/johnclark/Documents/Finances/Umbrella{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
    umbrella_check["account_name"] = "umbrella"
    savings_check = pd.read_csv("/Users/johnclark/Documents/Finances/Savings{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
    savings_check["account_name"] = "savings"
    primary_check = pd.read_csv("/Users/johnclark/Documents/Finances/Primary{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
    primary_check["account_name"] = "primary"
    platinum_card = pd.read_csv("/Users/johnclark/Documents/Finances/PlatinumCard{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
    platinum_card["account_name"] = "platinum"
    cashwise_card = pd.read_csv("/Users/johnclark/Documents/Finances/CashWiseCard{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
    cashwise_card["account_name"] = "cashwise"

    accounts_joined = accounts_joined.append(utility_check)
    accounts_joined = accounts_joined.append(umbrella_check)
    accounts_joined = accounts_joined.append(savings_check)
    accounts_joined = accounts_joined.append(primary_check)
    # accounts_joined = accounts_joined.append(platinum_card)
    # accounts_joined = accounts_joined.append(cashwise_card)

accounts_joined.reset_index(inplace=True, drop=True)

#assert len(accounts_joined) == len(utility_check) + len(umbrella_check) + len(savings_check) + len(primary_check)# + len(platinum_card) + len(cashwise_card)

accounts_joined["spending"] = accounts_joined["Amount"].apply(lambda x: x if x<0.0 else 0.0)
accounts_joined["income"] = accounts_joined["Amount"].apply(lambda x: x if x>0.0 else 0.0)

exclude_desc = {"FID BKG SVC LLC MONEYLINE 210402 X49800767 O0TLA JOHN P CLARK","RECURRING TRANSFER TO CLARK",
                "ONLINE TRANSFER TO CLARK J CHECKING XXXXXX7135", 
                "ONLINE TRANSFER TO CLARK J CHECKING XXXXXX9865", 
                "ONLINE TRANSFER TO CLARK J CHECKING XXXXXX9873", 
                "ONLINE TRANSFER TO CLARK J SAVINGS XXXXXX5751", 
                "ONLINE TRANSFER FROM CLARK J CHECKING XXXXXX7135",
                "ONLINE TRANSFER FROM CLARK J CHECKING XXXXXX9865",
                "ONLINE TRANSFER FROM CLARK J CHECKING XXXXXX9873",
                "ONLINE TRANSFER FROM CLARK J SAVINGS XXXXXX5751",
                "ONLINE TRANSFER FROM CLARK J",
                "ONLINE TRANSFER TO CLARK J"}
tithe_desc_starts = {"RURAL INDIA MINI", "BILL PAY Rural India Ministries ", "BILL PAY Church of the Open Door",
                     "CHECK # 136","BUSINESS TO BUSINESS ACH www.gracepres.co", 
                     "CHECK # 198",
                     "BILL PAY Church of the Resurrection",
                     "Flagstaff Christ", "FLAGSTAFF CHRIST",
                     "Mount Calvary"}
tax_desc_starts = {"CHECK # 132",
                   "CHECK # 131",
                   "FRANCHISE TAX BD CASTTAXRFD",
                   "IRS USATAXPYMT"}
shoffice_starts = {"CHECK # 129","DEPOSITED OR CASHED CHECK # 128","CASHED CHECK # 127"}
dirtbike_starts = {"CHECK # 195","CHECK # 196"}

accounts_joined["chase"] = accounts_joined.apply(lambda r: r["Amount"] if r["Description"].startswith("CHASE") else 0.0, raw=False, axis=1)
accounts_joined["tithe"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in tithe_desc_starts if r["Description"].startswith(td)]) > 0  else 0.0, raw=False, axis=1)
accounts_joined["taxes"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in tax_desc_starts if r["Description"].startswith(td)]) > 0  else 0.0, raw=False, axis=1)
accounts_joined["taxes_spend"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in tax_desc_starts if r["Description"].startswith(td)]) > 0 and r["Amount"] < 0.0  else 0.0, raw=False, axis=1)
accounts_joined["shoffice"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in shoffice_starts if r["Description"].startswith(td)]) > 0 and r["Amount"] < 0.0  else 0.0, raw=False, axis=1)
accounts_joined["dirtbikes"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in dirtbike_starts if r["Description"].startswith(td)]) > 0 and r["Amount"] < 0.0  else 0.0, raw=False, axis=1)
accounts_joined["apple"] = accounts_joined.apply(lambda r: r["Amount"] if r["Description"].startswith("APPLE INC. PAYROLL") else 0.0, raw=False, axis=1)

accounts_joined["base_spend_adjustment"] = 0.0
append_row(accounts_joined, "2022-05-09 00:00:00", "base_spend_adjustment", -692.20, description="Sig 365X") #
append_row(accounts_joined, "2022-05-09 00:00:00", "base_spend_adjustment", -2838.70, description="Stacato P") #

append_row(accounts_joined, "2022-02-27 00:00:00", "base_spend_adjustment", -2573.72, description="Avis mistake charge") #

append_row(accounts_joined, "2022-02-15 00:00:00", "base_spend_adjustment", -749.50, description="Alaska air") #
append_row(accounts_joined, "2022-02-15 00:00:00", "base_spend_adjustment", -749.50, description="Alaska air") #
append_row(accounts_joined, "2022-02-15 00:00:00", "base_spend_adjustment", -749.50, description="Alaska air") #
append_row(accounts_joined, "2022-02-15 00:00:00", "base_spend_adjustment", -749.50, description="Alaska air") #
append_row(accounts_joined, "2022-02-15 00:00:00", "base_spend_adjustment", -749.50, description="Alaska air") #
append_row(accounts_joined, "2022-02-15 00:00:00", "base_spend_adjustment", -749.50, description="Alaska air") #

append_row(accounts_joined, "2022-02-10 00:00:00", "base_spend_adjustment", -3458.58, description="Alaska turo car") # 
append_row(accounts_joined, "2022-02-05 00:00:00", "base_spend_adjustment", -5560.00, description="Alaska royal star train") #

append_row(accounts_joined, "2022-02-04 00:00:00", "base_spend_adjustment", -477.18, description="Alaska Grand Denali Lodge") #
append_row(accounts_joined, "2022-02-04 00:00:00", "base_spend_adjustment", -452.93, description="Alaska Grand Denali Lodge") #
append_row(accounts_joined, "2022-02-04 00:00:00", "base_spend_adjustment", -460.93, description="Alaska Grand Denali Lodge") #

append_row(accounts_joined, "2022-01-08 00:00:00", "base_spend_adjustment", -364.62, description="Sportsmans pelican case") #
append_row(accounts_joined, "2021-12-20 00:00:00", "base_spend_adjustment", -309.96, description="Southwest for Tara") #

append_row(accounts_joined, "2021-12-17 00:00:00", "base_spend_adjustment", -869.16, description="Suppressor") #
append_row(accounts_joined, "2021-12-17 00:00:00", "base_spend_adjustment", -2117.98, description="Herman miller chair") #

append_row(accounts_joined, "2021-11-26 00:00:00", "base_spend_adjustment", -375.47, description="Black Barts dinner") #
append_row(accounts_joined, "2021-11-08 00:00:00", "base_spend_adjustment", -107.99, description="Push to talk") #
append_row(accounts_joined, "2021-10-10 00:00:00", "base_spend_adjustment", -687.54, description="Anniversary El Quoquistador") #
append_row(accounts_joined, "2021-09-29 00:00:00", "base_spend_adjustment", -1916.00, description="10 yr Anniversary earings") #

append_row(accounts_joined, "2021-08-25 00:00:00", "base_spend_adjustment", -403.17, description="Moms Iphone") #
append_row(accounts_joined, "2021-08-25 00:00:00", "base_spend_adjustment", -2240.27, description="new Carpet final payment") #

append_row(accounts_joined, "2021-08-06 00:00:00", "base_spend_adjustment", -500.38, description="Watermans dinner for Tara and Mikes anniversary") #
append_row(accounts_joined, "2021-07-31 00:00:00", "base_spend_adjustment", -653.99, description="Papas ipad") #
append_row(accounts_joined, "2021-07-30 00:00:00", "base_spend_adjustment", -2619.25, description="Hannahs new laptop") #
append_row(accounts_joined, "2021-07-13 00:00:00", "base_spend_adjustment", -580.04, description="Sportsmans Hannahs scope") #
append_row(accounts_joined, "2021-07-11 00:00:00", "base_spend_adjustment", -952.03, description="Sportsmans Hannahs gun") #
append_row(accounts_joined, "2021-07-08 00:00:00", "base_spend_adjustment", -2072.00, description="New carpet") #
append_row(accounts_joined, "2021-06-24 00:00:00", "base_spend_adjustment", -378.59, description="El conquist") #
append_row(accounts_joined, "2021-06-20 00:00:00", "base_spend_adjustment", -250.00, description="Princess cruise") #

append_row(accounts_joined, "2021-06-18 00:00:00", "base_spend_adjustment", -375.00, description="Princess cruise") #
append_row(accounts_joined, "2021-06-18 00:00:00", "base_spend_adjustment", -500.00, description="Princess cruise") #
append_row(accounts_joined, "2021-06-18 00:00:00", "base_spend_adjustment", -250.00, description="Princess cruise") #

append_row(accounts_joined, "2021-03-29 00:00:00", "base_spend_adjustment", -880.00, description="hunting rifle") #

append_row(accounts_joined, "2021-02-23 00:00:00", "base_spend_adjustment", -1148.49, description="Dirt bike repair") #
append_row(accounts_joined, "2021-02-23 00:00:00", "base_spend_adjustment", -985.94, description="Dirt bike repair") #

append_row(accounts_joined, "2021-02-05 00:00:00", "base_spend_adjustment", -1182.10, description="Shoffice charge on cashwise") #
append_row(accounts_joined, "2021-02-05 00:00:00", "base_spend_adjustment", -46.25, description="Shoffice charge on cashwise") #
append_row(accounts_joined, "2021-02-05 00:00:00", "base_spend_adjustment", -231.14, description="Shoffice charge on cashwise") #
append_row(accounts_joined, "2021-02-05 00:00:00", "base_spend_adjustment", -61.71, description="Shoffice charge on cashwise") #
append_row(accounts_joined, "2021-02-05 00:00:00", "base_spend_adjustment", -60.97, description="Shoffice charge on cashwise") #
na_maps = {col_name:0.0 for col_name in accounts_joined.columns}
na_maps["Description"] = ""
accounts_joined.fillna(na_maps, inplace=True)


accounts_joined["filtered_amount"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in exclude_desc if r["Description"].startswith(td)]) == 0 else 0.0, raw=False, axis=1)
accounts_joined["filtered_income"] = accounts_joined.apply(lambda r: r["Amount"] if r["Amount"]>0.0 and len([1 for td in exclude_desc if r["Description"].startswith(td)]) == 0 else 0.0, raw=False, axis=1)

accounts_joined["full_spending"] = accounts_joined.apply(lambda r: r["Amount"] if r["Amount"]<0.0 and len([1 for td in exclude_desc if r["Description"].startswith(td)]) == 0 else 0.0, raw=False, axis=1)
accounts_joined["full_spending"] = accounts_joined["full_spending"] - accounts_joined["taxes_spend"]

accounts_joined["base_spending"] = accounts_joined.apply(lambda r: r["Amount"] if r["Amount"]<0.0 and len([1 for td in exclude_desc if r["Description"].startswith(td)]) == 0 else 0.0, raw=False, axis=1)
accounts_joined["base_spending"] = accounts_joined["base_spending"] - accounts_joined["tithe"] - accounts_joined["taxes_spend"] - accounts_joined["shoffice"] - accounts_joined["dirtbikes"] - accounts_joined["base_spend_adjustment"]

accounts_joined["comment"] = ""

notes = {"CHECK # 192":"Half cow 2020",
         "CHECK # 195":"Check to MR S for dirtbike",
         "CHECK # 196":"Check to MR S for dirtbike",
         "CHECK # 132":"Arizona Department of Revenue",
         "CHECK # 131":"United States Treasury",
         "CHECK # 129":"Master Mechanical",
         "CHECK # 130":"Hunting rifle",
         "CHECK # 126":"Evelyn enrollment",
         "CASHED CHECK # 127":"Dan shoffice reno",
         "DEPOSITED OR CASHED CHECK # 128":"Shoffice electrical",
         "FID BKG SVC LLC MONEYLINE 210402 X49800767 O0TLA JOHN P CLARK":"Transfer from fidelity for tithe",
         "FRANCHISE TAX BD CASTTAXRFD 040521 XXXXX8570 CLARK III, JOHN P & HA":"Franchise tax board CASTAX",
         "CHASE CREDIT CRD EPAY 210223 5147608953 JOHN CLARK III":"2365.00 to ACSTO",
         "CHASE CREDIT CRD EPAY 210824 5467211881 JOHN CLARK III":"600 FCS, 500.38 watermans CA, 653.99 apple, 2619.25 laptop",
         "CHASE CREDIT CRD EPAY 211022 5572610997 JOHN CLARK III":"4515.75 FCS",
         "CHASE CREDIT CRD EPAY 211226 5690175783 JOHN CLARK III":"869.16 supressor, 2117.98 chair, 2435 ATC",
         "CHECK # 134":"Ranch 39 cow",
         "CHECK # 136":"Church of the resurrection",
         "CHECK # 198":"Hope Cottage",
         "PURCHASE BANK CHECK OR DRAFT":"Suppressor tax stamp check",
         "CHECK # 137":"Check Mr S for rounds",
         "CHASE CREDIT CRD EPAY 220222 5796810515 JOHN CLARK III":"Alaska airline tikets, CA memorial flight and car, Turo Alaska car"}
for idx, row in accounts_joined.iterrows():
    if row["Description"] in notes:
        note = notes[row["Description"]]
        accounts_joined.loc[idx,"comment"] = note



In [13]:
import numpy as np
import datetime as dt

columns = ["Amount","income","filtered_amount","taxes","filtered_income", "apple", "spending","full_spending","base_spending","chase","tithe","taxes_spend","shoffice"]
sum_table = accounts_joined.groupby(pd.Grouper(key='Date', freq='1M')).sum()[columns]
sum_table.loc["Average"] = sum_table[columns].iloc[:-1,:].mean()
sum_table.loc["Totals"] = sum_table[columns].iloc[:-1,:].sum()

def red(s, props=''):
    return np.where(s < 0.0, props, '')
slice_1 = ["full_spending","base_spending"]
slice_2 = ["chase"]
slice_3 = ["tithe","taxes_spend","shoffice"]
df = sum_table.style.apply(red, props='color:red;', axis=0).set_properties(**{'background-color': '#ffffb3'}, subset=slice_1).set_properties(**{'background-color': '#e8e8e8'}, subset=slice_2).set_properties(**{'background-color': '#52ffff'}, subset=slice_3).format(precision=2)
#df = df.style.apply(red, props='color:red;', axis=0, subset=slice_2).set_properties(**{'background-color': '#3E85BD'}, subset=slice_2)
display(df)
#display(sum_table)





,Amount,income,filtered_amount,taxes,filtered_income,apple,spending,full_spending,base_spending,chase,tithe,taxes_spend,shoffice
Date,,,,,,,,,,,,,
2020-08-31 00:00:00,-3917.22,4805.57,-3207.22,0.00,4805.57,4094.56,-8722.79,-8012.79,-6012.54,-5697.62,-2000.25,0.00,0.00
2020-09-30 00:00:00,2620.29,12665.69,6822.01,0.00,12665.69,8189.11,-10045.40,-5843.68,-3043.43,0.00,-2800.25,0.00,0.00
2020-10-31 00:00:00,-826.61,22697.86,3675.11,0.00,20697.86,15598.70,-23524.47,-17022.75,-14222.50,-10212.09,-2800.25,0.00,0.00
2020-11-30 00:00:00,2506.31,13291.78,7208.03,0.00,13291.78,8589.46,-10785.47,-6083.75,-3283.50,0.00,-2800.25,0.00,0.00
2020-12-31 00:00:00,5068.62,67670.52,9670.34,0.00,47670.52,16068.18,-62601.90,-38000.18,-9249.68,-5877.48,-22800.50,0.00,0.00
2021-01-31 00:00:00,-3262.54,12768.21,1239.18,0.00,12768.21,8265.82,-16030.75,-11529.03,-8728.78,-5536.14,-2800.25,0.00,0.00
2021-02-28 00:00:00,-9186.00,12893.10,-4584.28,0.00,12893.10,8265.81,-22079.10,-17477.38,-8768.53,-7927.58,-2800.25,0.00,-2192.00
2021-03-31 00:00:00,-6911.47,32993.12,-2209.75,0.00,22993.12,8265.83,-39904.59,-25202.87,-9647.62,-5281.05,-12800.25,0.00,-1875.00
2021-04-30 00:00:00,66655.41,102292.20,21257.13,2469.00,51292.20,23632.51,-35636.79,-10112.07,-7311.82,-4580.39,-2800.25,-19923.00,0.00


## Show month

In [3]:
month = 5; year = 2022

start_date = '{}-{}-01'.format(year,month)
if month < 12:
    end_date = '{}-{}-01'.format(year,month+1)
else:
    end_date = '{}-01-01'.format(year+1)
view_table = accounts_joined[(accounts_joined["Date"] >= start_date) & (accounts_joined["Date"] < end_date)].sort_values("Date")

view_table



,Date,Amount,Unk,CheckNum,Description,account_name,spending,income,chase,tithe,...,taxes_spend,shoffice,dirtbikes,apple,base_spend_adjustment,filtered_amount,filtered_income,full_spending,base_spending,comment
623,2022-05-02,-2050.88,*,0.0,DOVENMUEHLE MTG MORTG PYMT 050122 1465323044 John Clark,umbrella,-2050.88,0.00,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,-2050.88,0.00,-2050.88,-2050.88,
624,2022-05-02,400.00,*,0.0,RECURRING TRANSFER FROM CLARK J REF #OP0F7NSBVW SAVINGS TO COVER GAS ELECTRIC WATER IN RENTAL,umbrella,0.00,400.00,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,400.00,400.00,0.00,0.00,
653,2022-05-02,-400.00,*,0.0,RECURRING TRANSFER TO CLARK J REF #OP0F7NSBVW PORTFOLIO CHECKING TO COVER GAS ELECTRIC WATER IN RENTAL,savings,-400.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,
654,2022-05-02,100.00,*,0.0,RECURRING TRANSFER FROM CLARK J CHECKING REF #OP0F7DYDRC XXXXXX9865,savings,0.00,100.00,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,100.00,100.00,0.00,0.00,
674,2022-05-02,-100.00,*,0.0,RECURRING TRANSFER TO CLARK J SAVINGS REF #OP0F7DYDRC XXXXXX5751,primary,-100.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,
671,2022-05-04,-114.41,*,0.0,ONLINE TRANSFER REF #IB0F8KBS8G TO WELLS FARGO CASH WISE VISA SIGNATURE CAR XXXXXXXXXXXX0560 ON 05/04/22,primary,-114.41,0.00,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,-114.41,0.00,-114.41,-114.41,
672,2022-05-04,-116.21,*,0.0,ONLINE TRANSFER REF #IB0F8KBPP6 TO PLATINUM CARD XXXXXXXXXXXX0729 ON 05/04/22,primary,-116.21,0.00,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,-116.21,0.00,-116.21,-116.21,
673,2022-05-04,-1000.00,*,0.0,BILL PAY Rural India Ministries RECURRING No Account Number ON 05-04,primary,-1000.00,0.00,0.00,-1000.0,...,0.0,0.0,0.0,0.00,0.0,-1000.00,0.00,-1000.00,0.00,
670,2022-05-05,-4094.32,*,0.0,CHASE CREDIT CRD EPAY 220503 5934913982 JOHN CLARK III,primary,-4094.32,0.00,-4094.32,0.0,...,0.0,0.0,0.0,0.00,0.0,-4094.32,0.00,-4094.32,-4094.32,
622,2022-05-09,0.01,*,0.0,INTEREST PAYMENT,umbrella,0.00,0.01,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.01,0.01,0.00,0.00,


## Show Tithe

In [ ]:
monthly_tithe = 1714.79
april_bonus = 16000
oct_monthly_tithe = 1714.79
oct_bonus = 11000
tithe_rows = accounts_joined[accounts_joined["Description"].apply(func=lambda r: True if len([1 for td in tithe_desc_starts if r.startswith(td)]) > 0 else False)]
current_month = tithe_rows["Date"].max().month
current_total_tithe = current_month * monthly_tithe
if current_month >= 4:
    current_total_tithe += round(april_bonus * 0.1, 2)
if current_month >= 10:
    current_total_tithe += round(oct_bonus * 0.1, 2)
print("Current tithe total: {} Current total paid: {} Difference: {}".format(current_total_tithe, tithe_rows["Amount"].sum(), round(current_total_tithe + tithe_rows["Amount"].sum(),2)))

tithe_rows



In [ ]:
eoy_primary_spend = end_of_year[end_of_year["Payment Method"].str.startswith("Primary ck acct")]
primary_spend = primary_check[primary_check["Amount"] < 0]

display(eoy_primary_spend.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount_value"))
display(primary_spend.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount"))



In [ ]:
cashwise_card

In [ ]:
platinum_card

## Net Worth

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

rows = []
rows.append({"date":"05/01/2019", "Wells cash":100260.40, "Wells credit":-1929.57, "chase credit":0.0,     "wells brokerage":20909.58, "fidelity":284004.18, "etrade":49422.75, "etrade stock":249679.65, 
"empower hannah":14593.43, "empower john":71455.34, "House value":1102000.00, "mortgage":-658366.50, "tithe":-24876.68})

rows.append({"date":"08/07/2019", "Wells cash":100689.28, "Wells credit":-329.74,  "chase credit":0.0,     "wells brokerage":20672.19, "fidelity":380078.53, "etrade":43028.98, "etrade stock":144660.80,
"empower hannah":14362.77, "empower john":78550.87, "House value":1102000.00, "mortgage":-653633.15, "tithe":-32218.83})

rows.append({"date":"11/06/2019", "Wells cash":58270.08, "Wells credit":0.00,     "chase credit":0.0,     "wells brokerage":20324.06, "fidelity":516164.05, "etrade":1941.79, "etrade stock":234587.60,
"empower hannah":15143.08, "empower john":93966.67, "House value":1102000.00, "mortgage":-650046.77, "tithe":-38781.82})

rows.append({"date":"02/10/2020", "Wells cash":53850.11, "Wells credit":0.00,     "chase credit":0.0,      "wells brokerage":20370.78, "fidelity":1015788.88, "etrade":1763.29, "etrade stock":204505.80,
"empower hannah":16206.57, "empower john":108802.06, "House value":1102000.00, "mortgage":-650046.77, "tithe":-42178.89})

rows.append({"date":"05/06/2020", "Wells cash":56692.33, "Wells credit":-269.45,   "chase credit":-1789.68, "wells brokerage":22450.00, "fidelity":847718.46, "etrade":2167.81,  "etrade stock":299564.10,
"empower hannah":13987.10, "empower john":98481.94, "House value":515000.00,  "mortgage":-411322.78, "tithe":-44378.09})

rows.append({"date":"08/06/2020", "Wells cash":46137.57, "Wells credit":0.00,     "chase credit":-2182.38,  "wells brokerage":24807.19, "fidelity":954744.14, "etrade":105017.35, "etrade stock":311637.24,
"empower hannah":16317.68, "empower john":123273.55, "House value":521693.00, "mortgage":-409280.11, "tithe":-47494.87})

rows.append({"date":"11/06/2020", "Wells cash":45168.12, "Wells credit":-214.66, "chase credit":-1663.29, "wells brokerage":25640.00, "fidelity":964813.58, "etrade":171603.54, "etrade stock":360115.46,
"empower hannah":16593.24, "empower john":133232.34, "House value":537331.00, "mortgage":-407220.79, "tithe":-49713.87})

rows.append({"date":"02/03/2021", "Wells cash":49157.78, "Wells credit":0.00,     "chase credit":-3773.34, "wells brokerage":28102.17, "fidelity":1212404.65, "etrade":144964.00, "etrade stock":329760.28,
"empower hannah":18813.23, "empower john":162068.47, "House value":515000,    "mortgage":-405144.70, "tithe":-40184.79})

rows.append({"date":"05/05/2021", "Wells cash":99890.90, "Wells credit":0.00,     "chase credit":-830.86,  "wells brokerage":31251.00, "fidelity":1324761.92, "etrade":45231.55, "etrade stock":399800.10,
"empower hannah":19662.01, "empower john":180939.49, "House value":624573.00, "mortgage":-403051.69, "tithe":-22053.44})

rows.append({"date":"08/06/2021", "Wells cash":92423.43, "Wells credit":0.00,    "chase credit":-5074.80,  "wells brokerage":28768.40, "fidelity":1407672.03, "etrade":16838.15, "etrade stock":472068.43,
"empower hannah":20508.15, "empower john":196907.71, "House value":694000.00, "mortgage":-400941.63, "tithe":-16856.93})

rows.append({"date":"11/09/2021", "Wells cash":83840.67, "Wells credit":0.00,   "chase credit":-3107.17,  "wells brokerage":28943.81, "fidelity":1480238.42, "etrade":14736.45, "etrade stock":585766.08,
"empower hannah":21213.00, "empower john":213646.38, "House value":694000.00, "mortgage":-398814.38, "tithe":-27061.92})

rows.append({"date":"02/03/2022", "Wells cash":80114.39, "Wells credit":0.00,    "chase credit":-2025.90,  "wells brokerage":27049.13, "fidelity":1398394.76, "etrade":16882.12, "etrade stock":685513.73,
"empower hannah":19732.53, "empower john":209110.90, "House value":746600,    "mortgage":-396669.80, "tithe":0.00})

rows.append({"date":"05/05/2022", "Wells cash":80610.18, "Wells credit":0.00,    "chase credit":0.00,      "wells brokerage":27031.94, "fidelity":1821888.93, "etrade":0.00,     "etrade stock":75539.83,
"empower hannah":19732.53, "empower john":202310.29, "House value":860300.00, "mortgage":-394507.75, "tithe":0.00})

net_worth = pd.DataFrame(rows)
net_worth["date"] = pd.to_datetime(net_worth["date"], format="%m/%d/%Y")
net_worth.set_index("date", drop=True, inplace=True)

nw_house_columns = ["Wells cash", "Wells credit", "chase credit", "wells brokerage", "fidelity", "etrade", "etrade stock","empower hannah", "empower john", "House value", "mortgage", "tithe"]
net_worth["total with house"] = net_worth[nw_house_columns].sum(axis=1)
nw_columns = ["Wells cash", "Wells credit", "chase credit", "wells brokerage", "fidelity", "etrade", "etrade stock","empower hannah", "empower john", "tithe"]
net_worth["total"] = net_worth[nw_columns].sum(axis=1)
net_worth["difference"] = 0.0
for row_idx in range(1,len(net_worth)):
    row_name = net_worth.iloc[row_idx].name
    net_worth.loc[row_name,"difference"] = net_worth.iloc[row_idx]["total"] - net_worth.iloc[row_idx-1]["total"]

display(net_worth)
fig = plt.figure()
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)
_ = net_worth[nw_columns].plot(legend=True, figsize=(30,30), ax=ax1)
_ = net_worth[["total with house","total"]].plot(legend=True, figsize=(30,30), ax=ax2)
_ = net_worth["difference"].plot.bar(legend=True, figsize=(30,30), ax=ax3)
ax1.grid(color='gray', linewidth=1, axis='both', alpha=0.5)
ax2.grid(color='gray', linewidth=1, axis='both', alpha=0.5)
ax2.axvline(x=dt.date(2022,2,7), color='red', linestyle='-', label='biked 75K')
ax2.axvline(x=dt.date(2022,4,6), color='green', linestyle='-', label='taxes 58K')
ax2.axvline(x=dt.date(2022,2,23), color='purple', linestyle='-', label='alaska 13K')
ax2.legend()
ax3.grid(color='gray', linewidth=1, axis='both', alpha=0.5)



In [ ]:
75+58+13